# MNIST 손글씨 분류기 - PyTorch, MLflow
- Fashion MNIST 이미지 분류 작업을 통해 PyTorch를 사용한 MLflow의 기능을 시연해 보겠습니다.  
  - 이미지 분류기로서 컨볼루션 신경망을 구축하고 다음 정보를 mlflow에 기록합니다:  
  - 훈련 지표: 훈련 손실 및 정확도.
  - 평가 지표: 평가 손실 및 정확도.
  - 훈련 설정: 학습 속도, 배치 크기 등.
  - 모델 정보: 모델 구조.
  - 저장된 모델: 학습 후 모델 인스턴스.

  

- 출처 : https://mlflow.org/docs/latest/deep-learning/pytorch/quickstart/pytorch_quickstart.html

### 패키지 설치

In [3]:
%pip install -q mlflow torchmetrics torchinfo

Note: you may need to restart the kernel to use updated packages.


##

In [16]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchinfo import summary
from torchmetrics import Accuracy
from torchvision import datasets
from torchvision.transforms import ToTensor

import mlflow

#### 데이터 준비하기 
이미 [0, 1]의 스케일로 사전 처리된 훈련 데이터 FashionMNIST를 torchvision에서 로드해 보겠습니다.   
그런 다음 데이터 세트를 torch.utils.data.Dataloader의 인스턴스로 래핑합니다.  

In [17]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [18]:
print(f"Image size: {training_data[0][0].shape}")
print(f"Size of training dataset: {len(training_data)}")
print(f"Size of test dataset: {len(test_data)}")

Image size: torch.Size([1, 28, 28])
Size of training dataset: 60000
Size of test dataset: 10000


배치 처리를 위해 데이터 집합을 DataLoader 인스턴스로 래핑합니다.   
DataLoader는 데이터 전처리를 위한 유용한 도구입니다.   

In [19]:
train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

### 모델 정의하기
- PyTorch 모델을 정의하려면 torch.nn.Module에서 서브클래싱하고 __init__를 재정의하여 모델 구성 요소를 정의하고 forward() 메서드를 재정의하여 정방향 전달 로직을 구현해야 합니다.
- 이미지 분류기로 2개의 컨볼루션 레이어로 구성된 간단한 컨볼루션 신경망(CNN)을 구축하겠습니다. 
- 모델 출력은 각 클래스(총 10개 클래스)의 로짓이 됩니다. 
- 로짓에 소프트맥스를 적용하면 클래스 간 확률 분포가 산출됩니다.

In [20]:
class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3),
            nn.ReLU(),
            nn.Conv2d(8, 16, kernel_size=3),
            nn.ReLU(),
            nn.Flatten(),
            nn.LazyLinear(10),  # 10 classes in total.
        )

    def forward(self, x):
        return self.model(x)


In [21]:
mlflow.set_experiment("mlflow-pytorch-quickstart")

<Experiment: artifact_location='file:///d:/%EA%B0%95%EC%9D%98%EC%9E%90%EB%A3%8C/MLflow/code/mlruns/581093473327566797', creation_time=1710547283371, experiment_id='581093473327566797', last_update_time=1710547283371, lifecycle_stage='active', name='mlflow-pytorch-quickstart', tags={}>

### Training Loop 구현
이제 기본적으로 데이터 집합을 반복하고 각 데이터 배치에 정방향 및 역방향 패스를 적용하는 학습 루프를 정의해 보겠습니다.

PyTorch는 수동 device 관리가 필요하므로 device 정보를 가져옵니다.

In [22]:
# Get cpu or gpu for training.
device = "cuda" if torch.cuda.is_available() else "cpu"

#### 트레이닝 함수를 정의합니다.

In [23]:
def train(dataloader, model, loss_fn, metrics_fn, optimizer, epoch):
    """Train the model on a single pass of the dataloader.

    Args:
        dataloader: an instance of `torch.utils.data.DataLoader`, containing the training data.
        model: an instance of `torch.nn.Module`, the model to be trained.
        loss_fn: a callable, the loss function.
        metrics_fn: a callable, the metrics function.
        optimizer: an instance of `torch.optim.Optimizer`, the optimizer used for training.
        epoch: an integer, the current epoch number.
    """
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        pred = model(X)
        loss = loss_fn(pred, y)
        accuracy = metrics_fn(pred, y)

        # Backpropagation.
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), batch
            step = batch // 100 * (epoch + 1)
            mlflow.log_metric("loss", f"{loss:2f}", step=step)
            mlflow.log_metric("accuracy", f"{accuracy:2f}", step=step)
            print(
                f"loss: {loss:2f} accuracy: {accuracy:2f} [{current} / {len(dataloader)}]"
            )

#### 각 에포크가 끝날 때 실행될 평가 함수를 정의합니다.

In [24]:
def evaluate(dataloader, model, loss_fn, metrics_fn, epoch):
    """Evaluate the model on a single pass of the dataloader.

    Args:
        dataloader: an instance of `torch.utils.data.DataLoader`, containing the eval data.
        model: an instance of `torch.nn.Module`, the model to be trained.
        loss_fn: a callable, the loss function.
        metrics_fn: a callable, the metrics function.
        epoch: an integer, the current epoch number.
    """
    num_batches = len(dataloader)
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            eval_loss += loss_fn(pred, y).item()
            eval_accuracy += metrics_fn(pred, y)

    eval_loss /= num_batches
    eval_accuracy /= num_batches
    mlflow.log_metric("eval_loss", f"{eval_loss:2f}", step=epoch)
    mlflow.log_metric("eval_accuracy", f"{eval_accuracy:2f}", step=epoch)

    print(f"Eval metrics: \nAccuracy: {eval_accuracy:.2f}, Avg loss: {eval_loss:2f} \n")

### Training
epochs 하이퍼 파라미터를 정의하고, 손실 함수를 선언하고, 모델을 생성하고,  optimizer를 인스턴스화 합니다.

In [25]:
epochs = 3
loss_fn = nn.CrossEntropyLoss()
metric_fn = Accuracy(task="multiclass", num_classes=10).to(device)
model = ImageClassifier().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

모든 것을 종합하여 훈련을 시작하고 MLflow에 정보를 기록해 보겠습니다.   
훈련 시작 시에는 훈련 및 모델 정보를 MLflow에 기록하고, 훈련 중에는 훈련 및 평가 메트릭을 기록합니다.   
모든 작업이 완료되면 학습된 모델을 기록합니다.  

In [26]:
with mlflow.start_run() as run:
    params = {
        "epochs": epochs,
        "learning_rate": 1e-3,
        "batch_size": 64,
        "loss_function": loss_fn.__class__.__name__,
        "metric_function": metric_fn.__class__.__name__,
        "optimizer": "SGD",
    }
    # Log training parameters.
    mlflow.log_params(params)

    # Log model summary.
    with open("model_summary.txt", "w") as f:
        f.write(str(summary(model)))
    mlflow.log_artifact("model_summary.txt")

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, metric_fn, optimizer, epoch=t)
        evaluate(test_dataloader, model, loss_fn, metric_fn, epoch=0)

    # Save the trained model to MLflow.
    mlflow.pytorch.log_model(model, "model")

Epoch 1
-------------------------------
loss: 2.301674 accuracy: 0.031250 [0 / 938]
loss: 2.274231 accuracy: 0.109375 [100 / 938]
loss: 2.219673 accuracy: 0.265625 [200 / 938]
loss: 2.158956 accuracy: 0.421875 [300 / 938]
loss: 1.950855 accuracy: 0.562500 [400 / 938]
loss: 1.593455 accuracy: 0.656250 [500 / 938]
loss: 1.280893 accuracy: 0.656250 [600 / 938]
loss: 0.988957 accuracy: 0.765625 [700 / 938]
loss: 0.931154 accuracy: 0.718750 [800 / 938]
loss: 0.876485 accuracy: 0.734375 [900 / 938]
Eval metrics: 
Accuracy: 0.72, Avg loss: 0.804336 

Epoch 2
-------------------------------
loss: 0.776549 accuracy: 0.765625 [0 / 938]
loss: 0.845168 accuracy: 0.703125 [100 / 938]
loss: 0.548310 accuracy: 0.843750 [200 / 938]
loss: 0.819213 accuracy: 0.687500 [300 / 938]
loss: 0.704526 accuracy: 0.671875 [400 / 938]
loss: 0.704366 accuracy: 0.781250 [500 / 938]
loss: 0.732565 accuracy: 0.718750 [600 / 938]
loss: 0.659362 accuracy: 0.765625 [700 / 938]
loss: 0.697894 accuracy: 0.718750 [800 / 938

마지막 단계로 모델을 다시 로드하고 추론을 실행해 보겠습니다.

In [27]:
logged_model = f"runs:/{run.info.run_id}/model"
loaded_model = mlflow.pyfunc.load_model(logged_model)

로드된 모델에 대한 입력은 numpy array 또는 pandas Dataframe이어야 하므로, tensor 를 명시적으로 numpy format 으로 캐스팅해야 합니다.

In [28]:
outputs = loaded_model.predict(training_data[0][0][None, :].numpy())